 # Wave Map Model Experiments

In [1]:
import datetime
import numpy as np
import pandas as pd

import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, MultiTaskLassoCV, SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR
from sklearn.model_selection import GridSearchCV

pd.options.plotting.backend = 'plotly'
np.random.seed(42)


In [2]:
# Load data
wave_data = pd.read_csv('data/wave_data.csv', header=[0,1], index_col=0, parse_dates=True)
weather_data = pd.read_csv('data/weather_data.csv', header=[0,1], index_col=0, parse_dates=True)


In [3]:
# Join data
data = pd.concat([wave_data, weather_data], axis=1, join='inner')


In [4]:
# Add squared wind speed
sq_dat = data.loc[:,(slice(None),'wind_speed')]**2
sq_dat.columns = pd.MultiIndex.from_tuples([(b, p+'_sq') for b, p in sq_dat.columns])
data = pd.concat([data, sq_dat], axis=1)


In [5]:
# Add lagging values
lagging_vars = ['10u', '10v', '2t', 'sp', 'wind_speed', 'wind_speed_sq']
lagging_times = [datetime.timedelta(hours=h) for h in range(1, 20)]
dat = [data]
for var in lagging_vars:
    for i, t in enumerate(lagging_times):
        var_dat = data.loc[:,(slice(None),var)]
        var_dat.index = var_dat.index + t
        var_dat.columns = pd.MultiIndex.from_tuples([(b, p+'_lag'+str(i+1)) for b, p in var_dat.columns])
        dat.append(var_dat)
data = pd.concat(dat, axis=1)


In [6]:
# Helper function
def describe_regression(pipe, coef_desc, target_desc):
    model = pipe.named_steps['regression']
    if len(model.coef_.shape) <= 1:
        return pd.DataFrame(
            np.append(model.coef_, model.intercept_).T,
            index=np.append(coef_desc, 'intercept'), columns=target_desc)
    else:
        return pd.DataFrame(
            np.append(model.coef_, model.intercept_[:,np.newaxis], axis=1).T,
            index=np.append(coef_desc, 'intercept'), columns=target_desc)


In [7]:
# Another helper function
def plot_prediction(pipe, X, y, func=lambda x: x.flatten()):
    y_pred = pipe.predict(X)
    fig = px.scatter(x=func(y), y=func(y_pred))
    fig.update_layout(title='Prediction vs true value')
    fig.update_xaxes(title='true value')
    fig.update_yaxes(title='prediction')
    fig.show()


 ## Train Test Split
 Use a randomly chosen one of the buoys as the test dataset.

In [8]:
num_buoys = data.columns.levshape[0]
test_buoy = 'pohjois-itaemeri'


In [9]:
test = data[test_buoy]
train = data.drop(test_buoy, axis=1)


 ## Simple Linear Regression
 Linear regression from Wind Vector, Temperature and Pressure to Wave Vector.
 No Temporal Information.

 Inputs:
 * 10u
 * 10v
 * 2t
 * sp


 Outputs:
 * wave_u
 * wave_v

In [10]:
# Build input and output matrices
input_vars = ['10u', '10v', '2t', 'sp']
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [11]:
# Train model
pipe = Pipeline([('scaler', StandardScaler()), ('regression', LinearRegression())])
pipe.fit(X_train, Y_train)
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Train accuracy of the model: 0.595


In [12]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))


Test accuracy of the model: 0.556


In [13]:
# Model coefficiens
describe_regression(pipe, input_vars, output_vars)


,wave_u,wave_v
10u,0.475594,0.115769
10v,0.007781,0.516111
2t,-0.038167,-0.007129
sp,-0.037664,-0.057895
intercept,0.233693,0.163399


 The linear regression gives the expected result: The are strong correlations between 10u-wave_u and 10v-wave_v.
 However, the accuracy of the model is not good.
 For some reason the train accuracy is lower than the test accuracy.

 ## Linear Regression with older inputs
 Linear regression from Wind Vector, Temperature and Pressure to Wave Vector.
 Each input variable is used four times, once with the current value, once lagging one hour and so on.

 Inputs:
 * 10u
 * 10v
 * 2t
 * sp
 * repeating from '_lag1' to '_lag3'


 Outputs:
 * wave_u
 * wave_v

In [14]:
# Build input and output matrices
base_vars = ['10u', '10v', '2t', 'sp']
input_vars = base_vars.copy()
for l in range(1, 4):
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [15]:
# Train model
pipe = Pipeline([('scaler', StandardScaler()), ('regression', LinearRegression())])
pipe.fit(X_train, Y_train)
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Train accuracy of the model: 0.641


In [16]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))


Test accuracy of the model: 0.604


In [17]:
# Model coefficiens
describe_regression(pipe, input_vars, output_vars)


,wave_u,wave_v
10u,0.152264,0.063697
10u_lag1,0.048364,0.071366
10u_lag2,-0.149572,-0.276660
10u_lag3,0.448331,0.249198
10v,0.004779,0.143884
10v_lag1,-0.071641,-0.004695
10v_lag2,0.144235,-0.115674
10v_lag3,-0.062859,0.531107
2t,-0.017008,0.044236
2t_lag1,-0.041182,-0.073338


 The regression model with lagging input variables shows increased accuracy.
 The largest influence on the model comes from the variables lagging the most, therefore longer lagging variables should be included.

 ## Linear Regression with even older inputs
 Linear regression from Wind Vector, Temperature and Pressure to Wave Vector.
 Each input variable is used nine times, once with the current value, once lagging one hour and so on.
 The goal here is to see how old the variables with the strongest correlation are.

 Inputs:
 * 10u
 * 10v
 * 2t
 * sp
 * repeating from '_lag1' to '_lag9'


 Outputs:
 * wave_u
 * wave_v

In [18]:
# Build input and output matrices
base_vars = ['10u', '10v', '2t', 'sp']
input_vars = base_vars.copy()
for l in range(1, 10):
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [19]:
# Train model
pipe = Pipeline([('scaler', StandardScaler()), ('regression', LinearRegression())])
pipe.fit(X_train, Y_train)
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Train accuracy of the model: 0.655


In [20]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))


Test accuracy of the model: 0.622


In [21]:
# Model coefficiens
describe_regression(pipe, input_vars, output_vars)


,wave_u,wave_v
10u,0.154774,0.046230
10u_lag1,0.013989,-0.012120
10u_lag2,0.087653,0.012538
10u_lag3,0.039182,0.004978
10u_lag4,0.058142,0.014094
10u_lag5,0.039508,0.016381
10u_lag6,0.026929,-0.025992
10u_lag7,0.035218,0.049622
10u_lag8,-0.026430,-0.061856
10u_lag9,0.089721,0.068774


 The model yields basically the same accuracy as when using less lagging variables.
 Of the lagging variables again, the oldest is the one with the strongest correlation.
 It is not clear from this result how much lag should be included.

 Next we try introducing regularization.

 ## Ridge Regression
 Ridge regression from Wind Vector, Temperature and Pressure to Wave Vector.
 Each input variable is used nine times, once with the current value, once lagging one hour and so on.
 The goal here is to see how old the variables with the strongest correlation are.

 Inputs:
 * 10u
 * 10v
 * 2t
 * sp
 * repeating from '_lag1' to '_lag9'


 Outputs:
 * wave_u
 * wave_v

In [22]:
# Build input and output matrices
base_vars = ['10u', '10v', '2t', 'sp']
input_vars = base_vars.copy()
for l in range(1, 10):
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [23]:
# Train model
alphas = np.logspace(-2, 5, num=20)
pipe = Pipeline([('scaler', StandardScaler()), ('regression', RidgeCV(alphas=alphas))])
pipe.fit(X_train, Y_train)
print('Chosen regularization parameter: alpha={:.3f}'.format(pipe.named_steps['regression'].alpha_))
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Chosen regularization parameter: alpha=48.329
Train accuracy of the model: 0.655


In [24]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))


Test accuracy of the model: 0.622


In [25]:
# Model coefficiens
describe_regression(pipe, input_vars, output_vars)


,wave_u,wave_v
10u,0.142101,0.042684
10u_lag1,0.039451,-0.002667
10u_lag2,0.070472,0.009756
10u_lag3,0.047349,0.004041
10u_lag4,0.054084,0.013206
10u_lag5,0.041685,0.010983
10u_lag6,0.031100,-0.002870
10u_lag7,0.022934,0.014199
10u_lag8,-0.014436,-0.039931
10u_lag9,0.083399,0.062901


 The regularization does not seem to influence the performance much.

 ## Lasso Regression
 Lasso Regression for selecting important variables.

 Inputs:
 * 10u
 * 10v
 * 2t
 * sp
 * repeating from '_lag1' to '_lag9'


 Outputs:
 * wave_u
 * wave_v

In [26]:
# Build input and output matrices
base_vars = ['10u', '10v', '2t', 'sp']
input_vars = base_vars.copy()
for l in range(1, 10):
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [27]:
# Train model
alphas = np.logspace(-2, 5, num=20)
pipe = Pipeline([('scaler', StandardScaler()), ('regression', MultiTaskLassoCV(alphas=alphas))])
pipe.fit(X_train, Y_train)
print('Chosen regularization parameter: alpha={:.3f}'.format(pipe.named_steps['regression'].alpha_))
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Chosen regularization parameter: alpha=0.010
Train accuracy of the model: 0.653


In [28]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))


Test accuracy of the model: 0.614


In [29]:
# Model coefficiens
coef = describe_regression(pipe, input_vars, output_vars)
coef


,wave_u,wave_v
10u,0.162133,0.043003
10u_lag1,0.004937,0.000906
10u_lag2,0.090687,0.012512
10u_lag3,0.041544,0.004853
10u_lag4,0.056993,0.006488
10u_lag5,0.034403,0.004144
10u_lag6,0.034929,0.004829
10u_lag7,0.003076,0.000563
10u_lag8,0.000000,0.000000
10u_lag9,0.077058,0.026465


In [30]:
# Unneeded variables
coef[(coef['wave_u'] == 0) & (coef['wave_v'] == 0)].index


Index([&#39;10u_lag8&#39;, &#39;10v_lag8&#39;, &#39;2t&#39;, &#39;2t_lag1&#39;, &#39;2t_lag2&#39;, &#39;2t_lag4&#39;,
       &#39;2t_lag5&#39;, &#39;2t_lag6&#39;, &#39;2t_lag7&#39;, &#39;sp_lag1&#39;, &#39;sp_lag2&#39;, &#39;sp_lag3&#39;,
       &#39;sp_lag4&#39;, &#39;sp_lag5&#39;, &#39;sp_lag6&#39;, &#39;sp_lag7&#39;, &#39;sp_lag8&#39;, &#39;sp_lag9&#39;],
      dtype=&#39;object&#39;)

 The lasso regression shows that Surface Pressure and Temperature can be dropped all together.
 Next we try lasso with only the wind components.

 ## Lasso Regression - Wind Only
 Lasso Regression for selecting which wind lag is important.

 Inputs:
 * 10u
 * 10v
 * repeating from '_lag1' to '_lag9'


 Outputs:
 * wave_u
 * wave_v

In [31]:
# Build input and output matrices
base_vars = ['10u', '10v']
input_vars = base_vars.copy()
for l in range(1, 10):
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [32]:
# Train model
alphas = np.logspace(-2, 5, num=20)
pipe = Pipeline([('scaler', StandardScaler()), ('regression', MultiTaskLassoCV(alphas=alphas))])
pipe.fit(X_train, Y_train)
print('Chosen regularization parameter: alpha={:.3f}'.format(pipe.named_steps['regression'].alpha_))
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Chosen regularization parameter: alpha=0.010
Train accuracy of the model: 0.648


In [33]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))


Test accuracy of the model: 0.610


In [34]:
# Model coefficiens
coef = describe_regression(pipe, input_vars, output_vars)
coef


,wave_u,wave_v
10u,0.175595,0.048166
10u_lag1,0.000000,0.000000
10u_lag2,0.087636,0.011842
10u_lag3,0.042225,0.004767
10u_lag4,0.056002,0.006162
10u_lag5,0.033991,0.003980
10u_lag6,0.035848,0.004863
10u_lag7,0.000365,0.000066
10u_lag8,0.000000,0.000000
10u_lag9,0.081620,0.028389


In [35]:
# Unneeded variables
coef[(coef['wave_u'] == 0) & (coef['wave_v'] == 0)].index


Index([&#39;10u_lag1&#39;, &#39;10u_lag8&#39;, &#39;10v_lag8&#39;], dtype=&#39;object&#39;)

 Not that much useful information.
 Why is the coefficient of the longest lag always that high?

 ## Lasso Regression - Wind Only - More Lag
 Lasso Regression for selecting which wind lag is important.

 Inputs:
 * 10u
 * 10v
 * repeating from '_lag2' '_lag4' to '_lag18'


 Outputs:
 * wave_u
 * wave_v

In [36]:
# Build input and output matrices
base_vars = ['10u', '10v']
input_vars = base_vars.copy()
for l in range(1, 10):
    input_vars.extend([v + '_lag' + str(l*2) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [37]:
# Train model
alphas = np.logspace(-2, 5, num=20)
pipe = Pipeline([('scaler', StandardScaler()), ('regression', MultiTaskLassoCV(alphas=alphas))])
pipe.fit(X_train, Y_train)
print('Chosen regularization parameter: alpha={:.3f}'.format(pipe.named_steps['regression'].alpha_))
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Chosen regularization parameter: alpha=0.010
Train accuracy of the model: 0.649


In [38]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))


Test accuracy of the model: 0.611


In [39]:
# Model coefficiens
coef = describe_regression(pipe, input_vars, output_vars)
coef


,wave_u,wave_v
10u,0.170329,0.046799
10u_lag10,0.018964,0.006332
10u_lag12,0.018491,0.007826
10u_lag14,0.000330,0.000154
10u_lag16,0.000000,0.000000
10u_lag18,0.011031,0.005829
10u_lag2,0.109882,0.014795
10u_lag4,0.100721,0.011654
10u_lag6,0.054962,0.008425
10u_lag8,0.032531,0.007921


In [40]:
# Unneeded variables
coef[(coef['wave_u'] == 0) & (coef['wave_v'] == 0)].index


Index([&#39;10u_lag16&#39;, &#39;10v_lag14&#39;, &#39;10v_lag16&#39;, &#39;10v_lag18&#39;], dtype=&#39;object&#39;)

 Here the strongest influence seems to be from the parameters lagging by 2 and 4 hours.
 Will keep 2 and 4 hour lag.

 ## Ridge Regression - Wind only
 Ridge regression from Wind Vector, Wind Speed and Wind Speed Squared
 Lag for 2 hours and 4 hours.

 Inputs:
 * 10u
 * 10v
 * 10u_lag2
 * 10v_lag2
 * 10u_lag4
 * 10v_lag4


 Outputs:
 * wave_u
 * wave_v

In [41]:
# Build input and output matrices
base_vars = ['10u', '10v']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [42]:
# Train model
alphas = np.logspace(-2, 5, num=20)
pipe = Pipeline([('scaler', StandardScaler()), ('regression', RidgeCV())])
pipe.fit(X_train, Y_train)
print('Chosen regularization parameter: alpha={:.3f}'.format(pipe.named_steps['regression'].alpha_))
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Chosen regularization parameter: alpha=10.000
Train accuracy of the model: 0.640


In [43]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))


Test accuracy of the model: 0.603


In [44]:
# Model coefficiens
describe_regression(pipe, input_vars, output_vars)


,wave_u,wave_v
10u,0.181888,0.100527
10u_lag2,0.035139,-0.116214
10u_lag4,0.294224,0.127650
10v,-0.027643,0.132095
10v_lag2,0.039646,0.067571
10v_lag4,0.005435,0.365886
intercept,0.233693,0.163399


 ## Ridge Regression - Wind only - More Features
 Ridge regression from Wind Vector, Wind Speed and Wind Speed Squared
 Lag for 2 hours and 4 hours.

 Inputs:
 * 10u
 * 10v
 * wind_speed
 * wind_speed_sq
 * 10u_lag2
 * 10v_lag2
 * wind_speed_lag2
 * wind_speed_sq_lag2
 * 10u_lag4
 * 10v_lag4
 * wind_speed_lag4
 * wind_speed_sq_lag4


 Outputs:
 * wave_u
 * wave_v

In [45]:
# Build input and output matrices
base_vars = ['10u', '10v', 'wind_speed', 'wind_speed_sq']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [46]:
# Train model
alphas = np.logspace(-2, 5, num=20)
pipe = Pipeline([('scaler', StandardScaler()), ('regression', RidgeCV())])
pipe.fit(X_train, Y_train)
print('Chosen regularization parameter: alpha={:.3f}'.format(pipe.named_steps['regression'].alpha_))
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Chosen regularization parameter: alpha=10.000
Train accuracy of the model: 0.659


In [47]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))
plot_prediction(pipe, X_test, Y_test, func=lambda x: np.sqrt(np.sum(x**2, axis=1)))


Test accuracy of the model: 0.627


In [48]:
# Model coefficiens
describe_regression(pipe, input_vars, output_vars)


,wave_u,wave_v
10u,0.132618,0.102788
10u_lag2,0.051085,-0.120780
10u_lag4,0.295503,0.126620
10v,-0.016676,0.132647
10v_lag2,0.042303,0.071433
10v_lag4,-0.015191,0.361553
wind_speed,-0.096793,-0.028199
wind_speed_lag2,-0.067065,0.026061
wind_speed_lag4,-0.060660,0.056297
wind_speed_sq,0.157682,-0.003454


 Including the wind speed and the squared wind speed does improve the performance.

 ## Support Vector Machine
 Support vector regression from Wind Vector, Wind Speed and Wind Speed Squared
 Lag for 2 hours and 4 hours.

 Inputs:
 * 10u
 * 10v
 * wind_speed
 * wind_speed_sq
 * 10u_lag2
 * 10v_lag2
 * wind_speed_lag2
 * wind_speed_sq_lag2
 * 10u_lag4
 * 10v_lag4
 * wind_speed_lag4
 * wind_speed_sq_lag4


 Outputs:
 * wave_u
 * wave_v

In [49]:
# Build input and output matrices
base_vars = ['10u', '10v', 'wind_speed', 'wind_speed_sq']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [50]:
# Train model
pipe_u = Pipeline([('scaler', StandardScaler()), ('regression', LinearSVR())])
pipe_v = Pipeline([('scaler', StandardScaler()), ('regression', LinearSVR())])
pipe_u.fit(X_train, Y_train[:,0])
pipe_v.fit(X_train, Y_train[:,1])
print('Chosen regularization parameter (u): C={:.3f}'.format(pipe_u.named_steps['regression'].C))
print('Chosen regularization parameter (v): C={:.3f}'.format(pipe_v.named_steps['regression'].C))
print('Train accuracy of the model (u): {:.3f}'.format(pipe_u.score(X_train, Y_train[:,0])))
print('Train accuracy of the model (v): {:.3f}'.format(pipe_v.score(X_train, Y_train[:,1])))


Chosen regularization parameter (u): C=1.000
Chosen regularization parameter (v): C=1.000
Train accuracy of the model (u): 0.654
Train accuracy of the model (v): 0.610


In [51]:
# Test model
print('Test accuracy of the model (u): {:.3f}'.format(pipe_u.score(X_test, Y_test[:,0])))
print('Test accuracy of the model (v): {:.3f}'.format(pipe_v.score(X_test, Y_test[:,1])))
coef = describe_regression(pipe, input_vars, output_vars)
coef


Test accuracy of the model (u): 0.684
Test accuracy of the model (v): 0.537


,wave_u,wave_v
10u,0.132618,0.102788
10u_lag2,0.051085,-0.120780
10u_lag4,0.295503,0.126620
10v,-0.016676,0.132647
10v_lag2,0.042303,0.071433
10v_lag4,-0.015191,0.361553
wind_speed,-0.096793,-0.028199
wind_speed_lag2,-0.067065,0.026061
wind_speed_lag4,-0.060660,0.056297
wind_speed_sq,0.157682,-0.003454


In [52]:
# Unneeded variables
coef[(coef['wave_u'] == 0) & (coef['wave_v'] == 0)].index


Index([], dtype=&#39;object&#39;)

 Here the strongest influence seems to be from the parameters lagging by 2 and 4 hours.
 Will keep 2 and 4 hour lag.

 ## Kernel Ridge Regression (Linear)
 **This model does not run, therefore it is commented out**

 It is unclear if the correlation between wind and wave height ist linear.
 Before experimenting with hand crafter features, we try kernelized ridge regression.

 Inputs:
 * 10u
 * 10v
 * 10u_lag2
 * 10v_lag2
 * 10u_lag4
 * 10v_lag4


 Outputs:
 * wave_u
 * wave_v

In [53]:
# Build input and output matrices
base_vars = ['10u', '10v']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u', 'wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [54]:
# Train model
# pipe = Pipeline([('scaler', StandardScaler()), ('regression', KernelRidge(kernel='linear'))])
# pipe.fit(X_train, Y_train)
# print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


In [55]:
# Test model
# print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))
# plot_prediction(pipe, X_test, Y_test, func=lambda x: np.sqrt(np.sum(x**2, axis=1)))


 ## Ridge Regression - Wave height only
 The correlation between speed and height seems to be stronger than the direction correlation.
 Try predicting the height independently of the direction.

 Inputs:
 * 10u
 * 10v
 * wind_speed
 * wind_speed_sq
 * 10u_lag2
 * 10v_lag2
 * wind_speed_lag2
 * wind_speed_sq_lag2
 * 10u_lag4
 * 10v_lag4
 * wind_speed_lag4
 * wind_speed_sq_lag4


 Outputs:
 * wave_height

In [56]:
# Build input and output matrices
base_vars = ['10u', '10v', 'wind_speed', 'wind_speed_sq']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['Wave height (m)']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [57]:
# Train model
alphas = np.logspace(-2, 5, num=20)
pipe = Pipeline([('scaler', StandardScaler()), ('regression', RidgeCV())])
pipe.fit(X_train, Y_train)
print('Chosen regularization parameter: alpha={:.3f}'.format(pipe.named_steps['regression'].alpha_))
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Chosen regularization parameter: alpha=10.000
Train accuracy of the model: 0.768


In [58]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))
plot_prediction(pipe, X_test, Y_test)


Test accuracy of the model: 0.698


In [59]:
describe_regression(pipe, input_vars, output_vars)


,Wave height (m)
10u,-0.012963
10u_lag2,-0.023399
10u_lag4,0.043801
10v,-0.034786
10v_lag2,0.035068
10v_lag4,-0.011634
wind_speed,-0.100969
wind_speed_lag2,-0.095364
wind_speed_lag4,0.019655
wind_speed_sq,0.320768


 ## SGD Regressor - Huber Loss - Wave height only
 Try another loss function.

 Inputs:
 * 10u
 * 10v
 * wind_speed
 * wind_speed_sq
 * 10u_lag2
 * 10v_lag2
 * wind_speed_lag2
 * wind_speed_sq_lag2
 * 10u_lag4
 * 10v_lag4
 * wind_speed_lag4
 * wind_speed_sq_lag4


 Outputs:
 * wave_height

In [60]:
# Build input and output matrices
base_vars = ['10u', '10v', 'wind_speed', 'wind_speed_sq']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['Wave height (m)']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [61]:
# Train model
pipe = Pipeline([('scaler', StandardScaler()), ('regression', SGDRegressor(loss='huber'))])
pipe.fit(X_train, Y_train)
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Train accuracy of the model: 0.763


In [62]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))
plot_prediction(pipe, X_test, Y_test)


Test accuracy of the model: 0.738


In [63]:
describe_regression(pipe, input_vars, output_vars)


,Wave height (m)
10u,-0.014500
10u_lag2,-0.003011
10u_lag4,0.032978
10v,-0.007245
10v_lag2,0.014125
10v_lag4,-0.021825
wind_speed,-0.047588
wind_speed_lag2,-0.088240
wind_speed_lag4,-0.003859
wind_speed_sq,0.267649


 ## SGD Regressor - Huber Loss - Wave U
 First part of SGD model for direction

 Inputs:
 * 10u
 * 10v
 * wind_speed
 * wind_speed_sq
 * 10u_lag2
 * 10v_lag2
 * wind_speed_lag2
 * wind_speed_sq_lag2
 * 10u_lag4
 * 10v_lag4
 * wind_speed_lag4
 * wind_speed_sq_lag4


 Outputs:
 * wave_u

In [64]:
# Build input and output matrices
base_vars = ['10u', '10v', 'wind_speed', 'wind_speed_sq']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_u']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [65]:
# Train model
pipe = Pipeline([('scaler', StandardScaler()), ('regression', SGDRegressor(loss='huber'))])
pipe.fit(X_train, Y_train)
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Train accuracy of the model: 0.655


In [66]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))
plot_prediction(pipe, X_test, Y_test)


Test accuracy of the model: 0.680


In [67]:
describe_regression(pipe, input_vars, output_vars)


,wave_u
10u,0.087999
10u_lag2,0.108302
10u_lag4,0.247223
10v,0.011079
10v_lag2,0.014892
10v_lag4,-0.027966
wind_speed,-0.070001
wind_speed_lag2,-0.077838
wind_speed_lag4,-0.068712
wind_speed_sq,0.139054


 ## SGD Regressor - Huber Loss - Wave V
 First part of SGD model for direction

 Inputs:
 * 10u
 * 10v
 * wind_speed
 * wind_speed_sq
 * 10u_lag2
 * 10v_lag2
 * wind_speed_lag2
 * wind_speed_sq_lag2
 * 10u_lag4
 * 10v_lag4
 * wind_speed_lag4
 * wind_speed_sq_lag4


 Outputs:
 * wave_v

In [68]:
# Build input and output matrices
base_vars = ['10u', '10v', 'wind_speed', 'wind_speed_sq']
input_vars = base_vars.copy()
for l in [2, 4]:
    input_vars.extend([v + '_lag' + str(l) for v in base_vars])
input_vars.sort()
output_vars = ['wave_v']

test_dropped = test.dropna()
X_test = test_dropped[input_vars].values
Y_test = test_dropped[output_vars].values

X_train = None
Y_train = None
for i, buoy in enumerate(train.columns.levels[0].drop(test_buoy)):
    buoy_dat = train[buoy].dropna()
    if X_train is None:
        X_train = buoy_dat[input_vars].values
        Y_train = buoy_dat[output_vars].values
    else:
        X_train = np.concatenate([X_train, buoy_dat[input_vars].values])
        Y_train = np.concatenate([Y_train, buoy_dat[output_vars].values])


In [69]:
# Train model
pipe = Pipeline([('scaler', StandardScaler()), ('regression', SGDRegressor(loss='huber'))])
pipe.fit(X_train, Y_train)
print('Train accuracy of the model: {:.3f}'.format(pipe.score(X_train, Y_train)))


Train accuracy of the model: 0.615


In [70]:
# Test model
print('Test accuracy of the model: {:.3f}'.format(pipe.score(X_test, Y_test)))
plot_prediction(pipe, X_test, Y_test)


Test accuracy of the model: 0.535


In [71]:
describe_regression(pipe, input_vars, output_vars)

,wave_v
10u,0.005811
10u_lag2,-0.018893
10u_lag4,0.094093
10v,0.085143
10v_lag2,0.122740
10v_lag4,0.237240
wind_speed,-0.045317
wind_speed_lag2,-0.052197
wind_speed_lag4,-0.017098
wind_speed_sq,0.088123
